<a href="https://colab.research.google.com/github/owenburton/DS-Unit-2-Applied-Modeling/blob/master/module1/assignment_applied_modeling_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 3, Module 1*

---


# Define ML problems

You will use your portfolio project dataset for all assignments this sprint.

## Assignment

Complete these tasks for your project, and document your decisions.

- [x] Choose your target. Which column in your tabular dataset will you predict? 
      'overall' (book rating)
- [x] Choose which observations you will use to train, validate, and test your model. And which observations, if any, to exclude. 
      Include all observations except missing reviews. 70/30 random, stratified train_test_split. Split the test into a bunch of validation sets and one final test set.
- [x] Determine whether your problem is regression or classification.
      Imbalanced multi-class classification (above 56% are 5-star ratings). Set class weights for your model.
- [ ] Choose your evaluation metric.
- [x] Begin to clean and explore your data.
      Using spacy. Help.
- [ ] Begin to choose which features, if any, to exclude. Would some features "leak" information from the future?
      Include 'reviewText'. Maybe include 'summary'. Possibly include 'reviewTime'.
